In [1]:
##Project 1 JN

#Most interested in:
""""
* Event Date - have certain planes become safer over the years?
* Purpose of Flight - are certain purposes more dangerous than others?
* Injury Severity - how severe were the injuries?
* Aircraft damage - how severe were the planes damaged?
* All columns related to injuries - most important factor is human lives, how do they fare in accidents?
* Weather Condition - what type of weather has been involved with crashes?
* Broad Phase of Flight - do certain planes not fare well in specific phases of flight?
* Report Status - do certain planes tend to have fault on the plane themselves?


Thoughts:
* Can we show that there are planes that have been in bad crashes but with minimal injuries or fatalaties? 
    If so, shows that plan is safe in the case of an accident.
* Do certain planes fare better than others in Weather Conditions?

Initial Tasks:
* DONE --- standardize column names
* DONE --- change date column to date format
* DONE --- change Inj cols with values like "Fatal(2)" to 'Fatal' b/c # fatal is taken care of in Total.Fatal.Injuries
* IN PROGRESS --- Analyze how many Null vales we have and decide how to handle

Possible Recommendations:
* which type of aircraft is safest (back it up with data related to #injuries/fatalaties and any growth in safety over time)
* which locations seem safer (see if there are areas that are more dangerous potentially due to weather)
* which purpose of flight tends to be safer (see if correlation between crashes and personal or commercial)

"""

In [2]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#import data and save to df
aviation_df = pd.read_csv('Data/Aviation_Data.csv')
state_codes_df = pd.read_csv('Data/USState_Codes.csv')

/Users/robertdaly/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
#Standardize column names
aviation_df = aviation_df.rename(columns = lambda x: x.replace(".", "_").lower())

#Convert event_date column to timestamp type
aviation_df['event_date'] = pd.to_datetime(aviation_df['event_date'])

In [5]:
aviation_df.head()

,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,...,purpose_of_flight,air_carrier,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,publication_date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.9222,-81.8781,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [6]:
aviation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_id                88889 non-null  object        
 1   investigation_type      90348 non-null  object        
 2   accident_number         88889 non-null  object        
 3   event_date              88889 non-null  datetime64[ns]
 4   location                88837 non-null  object        
 5   country                 88663 non-null  object        
 6   latitude                34382 non-null  object        
 7   longitude               34373 non-null  object        
 8   airport_code            50249 non-null  object        
 9   airport_name            52790 non-null  object        
 10  injury_severity         87889 non-null  object        
 11  aircraft_damage         85695 non-null  object        
 12  aircraft_category       32287 non-null  object

In [7]:
aviation_df['injury_severity'].head(30)

0      Fatal(2)
1      Fatal(4)
2      Fatal(3)
3      Fatal(2)
4      Fatal(1)
5     Non-Fatal
6      Fatal(4)
7     Non-Fatal
8     Non-Fatal
9     Non-Fatal
10    Non-Fatal
11    Non-Fatal
12    Non-Fatal
13     Fatal(1)
14     Fatal(1)
15     Fatal(2)
16    Non-Fatal
17     Fatal(3)
18    Non-Fatal
19    Non-Fatal
20    Non-Fatal
21    Non-Fatal
22    Non-Fatal
23     Incident
24     Fatal(2)
25     Fatal(8)
26     Fatal(1)
27    Non-Fatal
28    Non-Fatal
29    Non-Fatal
Name: injury_severity, dtype: object

In [8]:
#Analuze injury columns and notice that we have very weird data with only Max present
aviation_df[['injury_severity','total_fatal_injuries','total_serious_injuries','total_minor_injuries','total_uninjured']].describe()

,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured
count,77488.000000,76379.000000,76956.000000,82977.000000
mean,0.647855,0.279881,0.357061,5.325440
std,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,2.000000
max,349.000000,161.000000,380.000000,699.000000


In [9]:
#Find how many NAs in this column -> 2.7% of all entries have NA for injury_severity
aviation_df['injury_severity'].isna().sum()

#Change NAs in this column to Not Applicable for now. Can possibly change this later based on injury columns 
aviation_df.loc[aviation_df['injury_severity'].isna()] = 'Not Applicable'

#Check to confirm the replacement worked
aviation_df['injury_severity'].isna().sum()

0

In [10]:
#Change all values with Fatal() to just Fatal b/c counts are taken care of in total_fatalaties column
aviation_df.loc[aviation_df['injury_severity'].str.contains("Fatal\(.*\)")] = 'Fatal'

In [11]:
#Confirm that change was successful
aviation_df['injury_severity'].value_counts()

Non-Fatal         67357
Fatal             17826
Not Applicable     2459
Incident           2219
Minor               218
Serious             173
Unavailable          96
Name: injury_severity, dtype: int64

# Checkpoint
### At this point we have completed the following:
    1. Import libraries
    2. Import data
    3. Standardize column names
    4. Change event_date type to timestamp
    5. Clean up 'injury_severity' column by replaceing NAs with 'Not Applicable' 
        and removing counts b/c are represented in total_fatalities column